In [3]:
import pandas as pd
import numpy as np
import cv2
import random
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 数据预处理

In [6]:
raw_data = pd.read_csv('../data/train.csv',header=0)
data = raw_data.values
imgs = data[:, 1:]
labels = data[:, 0]
# 选取 2/3 数据作为训练集， 1/3 数据作为测试集
train_features, test_features, train_labels, test_labels = train_test_split(imgs, labels, test_size=0.33, random_state=23323)

print(train_features.shape)
print(test_features.shape)

(28140, 784)
(13860, 784)


# train

In [7]:
# 二值化
def binaryzation(img):
    cv_img = img.astype(np.uint8)
    cv2.threshold(cv_img, 50, 1, cv2.THRESH_BINARY_INV, cv_img)
    return cv_img

    cv2.threshold(cv_img, 50, 1, cv2.THRESH_BINARY_INV, cv_img)
这句代码中，cv_img是输入的784个pixel数字（0~255），50表示阈值，1表示最大值，cv2.THRESH_BINARY_INV表示二值化的类型。这句代码表示pixel数字大于50的部分，为1，小于50的部分，为0。

看一下经过二值化处理后是什么效果：

In [16]:
trainset[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [17]:
binaryzation(trainset[0])     # 图片二值化

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 0,

In [8]:
trainset, train_labels = train_features, train_labels

In [9]:
class_num = 10
feature_len = 784

In [11]:
# 存放先验概率
prior_probability = np.zeros(class_num)      
print(prior_probability.shape)
# 存放条件概率
conditional_probability = np.zeros((class_num, feature_len, 2))    
print(conditional_probability.shape)

(10,)
(10, 784, 2)


具体文章参考这一篇[机器学习通俗入门-朴素贝叶斯分类器](http://blog.csdn.net/TaiJi1985/article/details/73657994)

$x^{(i)}$ 为一个28维的向量表示第i个样本， $y^{(i)}$ 为标注的类别。我们求解的目标是：

$$f = \underset{j}{arg maxP} (y^{(i)} = j \mid x^{(i)}) $$

简单说就是计算$p (y^{(i)} = 0 \mid x^{(i)}) $，$p (y^{(i)} = 1 \mid x^{(i)}) $ ... $p (y^{(i)} = 9 \mid x^{(i)}) $，从中找出一个最大的，如果从属于第j个类的概率最大，那么就认为这张图片从属于j这个类。


In [23]:
# 计算先验概率及条件概率
for i in range(len(train_labels)):
    img = binaryzation(trainset[i])     # 图片二值化
    label = train_labels[i]

    prior_probability[label] += 1      # 每个label的图片各有多少个

    for j in range(feature_len):
        conditional_probability[label][j][img[j]] += 1  
        # img[j]表示在像素点j上的值。如果是0，就会给第一个位置+1，如果是1，会给第二个位置+1
        # 比如下面的conditional_probability[0][0]，结果是[0, 2711]。
        # 说明在img中，标签为0的样本中，像素点为0的对应位置，在img中分别为0或1的数量

In [39]:
prior_probability

array([ 2711.,  3197.,  2828.,  2897.,  2751.,  2565.,  2769.,  2964.,
        2654.,  2804.])

把上面的循环拆解分析一下，这里取第一个训练样本：


In [25]:
img1 = binaryzation(trainset[0])     # 图片二值化, 784个像素（feature）中，要么是0，要么是1
label1 = train_labels[0]
label1

9

In [26]:
img1[0]

1

In [24]:
print(conditional_probability[0][0]) # 

array([    0.,  2711.])

说明在img中，标签为0的样本中，像素点为500的对应位置，在img中为0的样本数量是199，为1的样本数量是2512

In [27]:
print(conditional_probability[0][500]) # 

[  199.  2512.]


下面之所以将概率归到[1.10001]，是因为上面所有关于概率的部分都是直接用样本数量，而不是实际的概率来记录的。这么做应该是为了在工程上解决内存，但是这种工程上的优化，对于理解书中的公式造成了影响。

而且下面计算概率的时候有点问题：
            probalility_0 = (float(pix_0)/float(pix_0+pix_1))*1000000 + 1
分母部分是，属于i类(0~9)的图像中，像素j的数量……对啊，这个像素j的数量其实就是pix_0和pix_1的和，即属于i类的图像的数量。看来这里没问题，是我想多了。

In [28]:
# 将概率归到[1.10001]
for i in range(class_num):
    for j in range(feature_len):

        # 经过二值化后图像只有0，1两种取值
        pix_0 = conditional_probability[i][j][0]  # 属于i类(0~9)的图像中，像素j(0~783)为0的数量
        pix_1 = conditional_probability[i][j][1]  # 属于i类(0~9)的图像中，像素j(0~783)为1的数量

        # 计算0，1像素点对应的条件概率
        probalility_0 = (float(pix_0)/float(pix_0+pix_1))*1000000 + 1
        probalility_1 = (float(pix_1)/float(pix_0+pix_1))*1000000 + 1

        conditional_probability[i][j][0] = probalility_0
        conditional_probability[i][j][1] = probalility_1

In [29]:
conditional_probability[0][0]

array([  1.00000000e+00,   1.00000100e+06])

得到了prior_probability和conditional_probability，这就算是训练结束了。

# test (predict)


In [34]:
# 为了加快预测速度，这里直接取100个测试样本

testset = test_features[:100]
testset.shape

(100, 784)



$$p (y^{(i)} = j \mid x_{k}^{(i)})  = \frac{p (x_{k}^{(i)} \mid y^{(i)} = j) \cdot p(y^{(i)} = j)}{p(x_{k}^{(i)})}$$

$p (y^{(i)} = j \mid x_{k}^{(i)}) $中，$y^{(i)} = j$表示从属于哪一类，$x_{k}^{(i)}$表示哪一个像素点。

下面calculate_probability函数就是在计算分子部分。

`probability *= int(conditional_probability[label][i][img[i]])`

这行代码中：
- probability表示先验概率 $p(y^{(i)} = j)$
- `conditional_probability[label][i][img[i]]`表示 $p (x_{k}^{(i)} \mid y^{(i)} = j) $


In [35]:
# 计算不同标签下，testdata的概率
def calculate_probability(img, label):
    probability = int(prior_probability[label])

    for i in range(len(img)):
        probability *= int(conditional_probability[label][i][img[i]])

    return probability

In [36]:
predict = []

for img in testset:

    # 图像二值化
    img = binaryzation(img)

    max_label = 0
    max_probability = calculate_probability(img, 0)

    for j in range(1, 10):
        probability = calculate_probability(img, j)

        if max_probability < probability:
            max_label = j
            max_probability = probability

    predict.append(max_label)

In [37]:
test_predict = np.array(predict)

In [38]:
score = accuracy_score(test_labels[:100], test_predict)
score

0.76000000000000001

# 重构朴素贝叶斯算法

![](https://pic1.zhimg.com/v2-e17426fd0627560f1fc82118dd1d5d14_r.jpg)

朴素贝叶斯认为所有特征都是独立的，然后得出一个样本出现的概率使其所有特征出现概率的联乘。

首先求每一个标签的先验概率：

In [46]:
class_num = 10
feature_len = 784

# 存放每个label的数量
class_number = np.zeros(class_num)      

# 存放先验概率
prior_probability = np.zeros(class_num)      
print(prior_probability.shape)
# 存放条件概率
conditional_probability = np.zeros((class_num, feature_len, 2))    
print(conditional_probability.shape)

(10,)
(10, 784, 2)


In [47]:
# 计算先验概率
for i in range(len(train_labels)):
    img = binaryzation(trainset[i])     # 图片二值化
    label = train_labels[i]

    class_number[label] += 1      # 每个label的图片各有多少个


In [48]:
class_number/len(train_labels)

array([ 0.09633973,  0.11361052,  0.10049751,  0.10294954,  0.09776119,
        0.09115139,  0.09840085,  0.10533049,  0.09431414,  0.09964463])

In [49]:
class_number

array([ 2711.,  3197.,  2828.,  2897.,  2751.,  2565.,  2769.,  2964.,
        2654.,  2804.])

In [50]:
prior_probability = class_number / len(train_labels)

计算条件概率: 

$$p (X^{(i)} = a_{jl} \mid Y = c_k)$$

在标签为$c_k$的前提下，样本x的第$j$个特征（像素点）的第$l$个值（经过二值化处理，这里的$l$只有0或1两种可能）。conditional_probability的维度是`(10, 784, 2)`，最后的那个2，指的就是每个特征可以取的值。如果不做二值化处理，那么每个像素点应该有256种取值。

In [66]:
# 条件概率
conditional_probability = np.zeros((class_num, feature_len, 2))    

for i in range(len(train_labels)):
    img = binaryzation(trainset[i])     # 图片二值化
    label = train_labels[i]
    for j in range(feature_len):
        conditional_probability[label][j][img[j]] += 1   # 这里只得到a_jl的数量

In [59]:
conditional_probability[0][500] 

array([  199.,  2512.])

In [65]:
class_number[0]

2711.0

In [60]:
conditional_probability[0][500] / class_number[0]

array([ 0.07340465,  0.92659535])

In [69]:
conditional_probability_fraction = np.zeros((class_num, feature_len, 2))    

for i in range(len(train_labels)):
    label = train_labels[i]
    for j in range(feature_len):
        conditional_probability_fraction[label][j] = conditional_probability[label][j] / class_number[label]

In [70]:
conditional_probability_fraction[0][500]

array([ 0.07340465,  0.92659535])

发现上面如果分开两循环写的话冗长，这里还是应该写在一起：

In [ ]:
# 计算先验概率及条件概率
for i in range(len(train_labels)):
    img = binaryzation(trainset[i])     # 图片二值化
    label = train_labels[i]

    class_number[label] += 1      # 每个label的图片各有多少个
    prior_probability = class_number / len(train_labels)

    for j in range(feature_len):
        conditional_probability[label][j][img[j]] += 1  
        # 在所有训练样本中，标签=0的样本中，像素点=0的对应位置上，一共有多少个样本是0，一共有多少个样本是1
        
# 推荐概率        
for i in range(class_num):
    for j in range(feature_len):
        conditional_probability[label][j] = conditional_probability[label][j] / class_number[label]

上面就算完成了第一步，计算完了先验概率和条件概率。接下来第二步对测试集进行预测:

In [71]:
testset.shape

(100, 784)

In [72]:
# 写一个函数来计算每一个标签下，对应的概率
def calculate_probability(img, label):
    probability = prior_probability[label] # 先验概率

    # 对每一个像素点进行迭代，计算在laebl固定的情况下，每一个像素点的概率，然后联乘
    for i in range(len(img)):
        probability *= conditional_probability[label][i][img[i]] 
        # [i]表示一个测试样本中，第i个像素点
        # img[i]表示一个测试样本中，第i个像素点是0还是1

    return probability

In [73]:
predict = []

for img in testset:
    img = binaryzation(img)
    
    max_label = 0
    max_probability = calculate_probability(img, 0)
    
    for j in range(1, 10):
        probability = calculate_probability(img, j)
        
        if max_probability < probability:
            max_label = j
            max_probability = probability

    predict.append(max_label)

/Users/xu/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:7: RuntimeWarning: overflow encountered in double_scalars
/Users/xu/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:7: RuntimeWarning: invalid value encountered in double_scalars


看来确实是这样，为了防止溢出，所以源代码里才一直用数量代替。

看来这个算法不需要我改了，源代码其实已经考虑了溢出的问题。